## Transfer Learning with VGG-16 CNN + AUG LB 0.1712

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

root_dir = Path('C:/Users/sinjy/jupyter_notebook/datasets')
data_dir = root_dir / 'kaggle_datasets' / 'Iceberg'
predict_dir = root_dir / 'kaggle_predict'

os.listdir(data_dir)

['sample_submission.csv', 'test.json', 'train.json']

## Transfer Learning with VGG-16
## data augmentation

## Load data

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [3]:
train = pd.read_json(data_dir / 'train.json')
target_train = train['is_iceberg']
test = pd.read_json(data_dir / 'test.json')

In [6]:
target_train = train['is_iceberg']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
X_angle = train['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle = test['inc_angle']

In [7]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)

In [8]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [14]:
from matplotlib import pyplot
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input	

### Data Augmentation

In [15]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64
gen = ImageDataGenerator(horizontal_flip=True,
                        vertical_flip=True, 
                        width_shift_range=0.,
                        height_shift_range=0.,
                        channel_shift_range=0.,
                        zoom_range=0.2,
                        rotation_range=10)

In [16]:
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=55)
    genX2 = gen.flow(X1, X2, batch_size=batch_size, seed=55)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[1]], X1i[1]

In [17]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [ ]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name='angle')
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                      input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    